In [1]:
import numpy as np
import pandas as pd
import spacy

# import stanza as st
# import simplemma

import re
import string
import fasttext as ft
import translators as ts
import itertools
ft_model = ft.load_model("lid.176.bin")
dir_ = 'BIG_UKR/'
name_uk = dir_+'country_heads_topic_ukraine.xlsx'
name = dir_+'country_heads_topic_all.xlsx'

/home/kwaks/gejopandas/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-08-07 23:49:37.313158: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-08-07 23:49:37.316022: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-07 23:49:37.316031: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Using Poland server backend.


In [2]:
from tqdm.notebook import tqdm
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text
preprocessor = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoderLabse = hub.KerasLayer("https://tfhub.dev/google/LaBSE/2")

def normalization(embeddings):
    norms = np.linalg.norm(embeddings, 2, axis=1, keepdims=True)
    return embeddings/norms

def val_ct_dict(obj):
    unique, counts = np.unique(obj, return_counts=True)
    val_ct = dict(zip(unique, counts))
    val_ct = dict(reversed(sorted(val_ct.items(), key=lambda item: item[1])))
    return val_ct

def embedd(sentence):
    if isinstance(sentence, str):
        embedding = tf.constant([sentence])
    else:
        embedding = tf.constant(sentence)
    embedding = encoderLabse(preprocessor(embedding))["default"]
    embedding = normalization(embedding)
    return embedding

def embedd_list(list_obj):
    embedded = np.zeros((len(list_obj), 768))
    with tqdm(total=len(list_obj)) as pbar:
        for i in range(len(list_obj)):
            embedded[i] = embedd(list_obj[i])
            pbar.update(1)
    return embedded

def val_ct_dict(obj):
    unique, counts = np.unique(obj, return_counts=True)
    val_ct = dict(zip(unique, counts))
    val_ct = dict(reversed(sorted(val_ct.items(), key=lambda item: item[1])))
    return val_ct

2022-08-07 23:49:39.778771: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-07 23:49:39.778787: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-07 23:49:39.778798: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Kwakuter): /proc/driver/nvidia/version does not exist
2022-08-07 23:49:39.778940: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
punctation = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '.',
           '/', ':', ';', '=', '?', '@', '[', '\\', ']', '^', 
           '`', '{', '|', '}', '~', '»', '«', '“', '”']
punct_pattern = re.compile("[" + re.escape("".join(punctation)) + "]")
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [4]:
file = pd.ExcelFile(name_uk)
countries = file.sheet_names

In [5]:
# all_langs = ['ar', 'be', 'bs', 'ca', 'cs', 'cy', 'da', 'de', 'ds', 'el', 'en',
#        'eo', 'es', 'eu', 'fi', 'fr', 'ga', 'hr', 'hu', 'hy', 'id', 'it',
#        'ja', 'km', 'ko', 'lb', 'ls', 'lt', 'lv', 'mk', 'mt', 'nl', 'nn',
#        'no', 'pl', 'pt', 'ro', 'ru', 'sh', 'sk', 'sl', 'sr', 'sv', 'th',
#        'uk', 'vi', 'zh']
# # all_langs

In [7]:
# !python -m spacy download en_core_web_lg
# !python -m spacy download de_core_news_lg
# !python -m spacy download nl_core_news_lg
# !python -m spacy download da_core_news_lg
# !python -m spacy download fr_core_news_lg
# !python -m spacy download it_core_news_lg
# !python -m spacy download pl_core_news_lg
# !python -m spacy download es_core_news_lg
# !python -m spacy download pt_core_news_lg
# !python -m spacy download ru_core_news_lg
# !python -m spacy download ro_core_news_lg
# !python -m spacy download fi_core_news_lg
# !python -m spacy download lt_core_news_lg
# !python -m spacy download sv_core_news_lg
# !python -m spacy download el_core_news_lg
# !python -m spacy download hr_core_news_lg
# !python -m spacy download ca_core_news_lg
# !python -m spacy download mk_core_news_lg

# !python -m spacy download uk_core_news_trf
# ukr-models/uk_core_news_trf

In [9]:
lang_frame = pd.read_excel(open(name, 'rb'),sheet_name='PL.1')
fix_names = lang_frame.columns

In [10]:
class Language_Models():
    culture_langs = {'AT': {'de': 207, 'en': 29, 'uk': 1, 'nl': 1, 'km': 1, 'fr': 1, 'es': 1},
                    'BE': {'nl': 157, 'fr': 154, 'en': 66, 'de': 47, 'th': 1, 'eu': 1, 'es': 1},
                    'BG': {'en': 5, 'lt': 1},
                    'CY': {'el': 17, 'en': 2},
                    'CZ': {'cz': 2137},   
                    'EE': {'en': 176, 'uk': 8, 'pl': 2, 'ru': 1, 'nl': 1, 'lv': 1, 'ja': 1, 'it': 1, 'fr': 1, 'de': 1},
                    'ES': {'es': 391, 'en': 4, 'fr': 2, 'zh': 1},
                    'FI': {'fi': 182, 'en': 116, 'sv': 56, 'no': 4, 'fr': 2, 'de': 2, 'es': 1},
                    'FR': {'fr': 106, 'it': 1, 'de': 1},
                    'GR': {'el': 20, 'en': 4, 'th': 1, 'fr': 1},
                    'IE': {'en': 110, 'ga': 12, 'de': 4, 'sk': 1, 'nl': 1, 'ko': 1},
                    'IT': {'it': 131, 'es': 1},
                    'LT': {'en': 169, 'uk': 8, 'de': 5, 'ru': 2, 'nl': 2, 'lt': 2, 'be': 2, 'hy': 1, 'fr': 1, 'da': 1},
                    'LU': {'fr': 105, 'en': 32, 'lb': 7, 'ru': 6, 'zh': 1, 'uk': 1, 'es': 1, 'de': 1},
                    'LV': {'lv': 579, 'en': 215, 'lt': 12, 'ja': 12, 'eu': 10, 'uk': 8, 'pl': 7, 'fr': 7, 'de': 7, 'nl': 6, 'ru': 4, 'es': 4, 'sr': 2, 'sk': 2, 'pt': 2, 'id': 2, 'th': 1, 'ro': 1, 'mk': 1, 'it': 1, 'hu': 1, 'hr': 1, 'el': 1},
                    'MT': {'en': 133, 'mt': 2, 'fi': 1, 'de': 1},
                    'NL': {'nl': 142, 'en': 9, 'fr': 3, 'pl': 1, 'es': 1},
                    'PL': {'pl': 2109, 'en': 87, 'de': 16, 'pt': 8, 'uk': 7, 'fr': 7, 'es': 7, 'nl': 4, 'lt': 3, 'eo': 3, 'zh': 2, 'sv': 2, 'sl': 2, 'ja': 2, 'it': 2, 'ru': 1, 'lv': 1, 'hu': 1, 'cs': 1},
                    'SI': {'sl': 549, 'en': 91, 'pl': 24, 'sr': 20, 'hr': 15, 'de': 9, 'it': 8, 'bs': 8, 'sh': 6, 'cs': 5, 'ru': 3, 'fr': 3, 'eo': 3, 'zh': 2, 'uk': 2, 'nl': 2, 'lt': 2, 'ar': 2, 'sk': 1, 'ro': 1, 'pt': 1, 'nn': 1, 'ls': 1, 'id': 1, 'hu': 1, 'es': 1, 'cy': 1},
                    'SK': {'en': 62, 'uk': 1, 'sk': 1, 'pl': 1, 'fr': 1, 'cs': 1, 'ca': 1},
                    'RO': {'en': 80, 'ro': 12, 'fr': 4, 'uk': 3, 'es': 2, 'pl': 1, 'de': 1},
                    'GB': {'en': 858, 'nl': 2, 'ja': 2, 'fr': 2, 'vi': 1, 'ru': 1, 'es': 1, 'ds': 1, 'de': 1, 'ca': 1}}
    
    def __init__(self, country):
        self.culture = country[:2]
        self.models = dict([ [cul, self.load_nlp_core(cul.lower())] for cul in self.culture_langs[self.culture].keys()])
    
    def get_dominant_language(self):
        return list(self.culture_langs[self.culture[:2]].keys())[0]
        
    def load_nlp_core(self, lang):
        try:
            print(f'{lang}_core_web_lg')
            if lang == 'en':
                core = spacy.load('en_core_web_lg')
            else:
                core = spacy.load(f'{lang}_core_news_lg')
            return core
        except:
            print(f'no {lang} nlp core')
            return None

    def get_nlp_core(self, lang):
        try:
            return self.models[lang]
        except:
            print(f'no {lang} nlp core')
            return None
        
    def get_stopwords(self, lang):
        try:
            return self.get_nlp_core(lang).Defaults.stop_words
        except:
            print(f'no {lang} nlp stopwords')
            return None


In [174]:
emotions_list = ["fear", "anger", "anticipation", "trust", "surprise", "sadness", "disgust", "joy"]

def get_sentiment_vec(twitt_lem):
# for twitt_lem, text in zip(lang_frame_uk.lemm_sentences,lang_frame_uk.text):
    sentences = twitt_lem.split('; ')
    for utterance in sentences:

        sentiment = []
        for word in utterance.split(', '):

            e1 = embedd(word)
            score = np.matmul(e1, np.transpose(en_index_emb))
            if max(score[0])>0.8:
                sentiment_tag = en_sent_lex.iloc[np.argmax(score[0])]
#                 print(word, sentiment_tag.lem, sentiment_tag.val,max(score[0]))
                emotions = sentiment_tag.val.split(',')
                sentiment.append(emotions)
        sentiment = np.array([item for sublist in sentiment for item in sublist])
        sentiment = val_ct_dict(sentiment)
        if sentiment.get("positive") and sentiment.get("negative"):
            vibe = (sentiment["positive"] - sentiment["negative"])/(sentiment["positive"] + sentiment["negative"])*100
            del sentiment["positive"]
            del sentiment["negative"]
        elif sentiment.get("positive"):
            vibe = 1
            del sentiment["positive"]
        elif sentiment.get("negative"):
            vibe = -1
            del sentiment["negative"]

        sentiment = dict([(i,j/sum(list(sentiment.values())) ) for i, j in sentiment.items()])
        sentiment_ret = [ sentiment.get(s) if sentiment.get(s) else 0 for s in emotions_list]
        return sentiment_ret


In [74]:
def process(text,lang_model):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(emoji_pattern, '', text)
    text = re.sub(r'\d+\/\d+', '', text)
    text = re.sub(r' {2,}', ' ', text)
    text = re.sub(r' $', '', text)
    text = re.sub(r'U/+.{5}', '', text)
    text = re.sub(r'\-+', '', text)
    sentences = re.split('\.{3} |\. |\! ',text)
    langs = []
    for x in range(len(sentences)):
        sentences[x] = re.sub(punct_pattern, '', sentences[x])
        sentences[x] = re.sub(r'\n', '', sentences[x])
    sentences = [x for x in sentences if not x.isspace() or len(x)>0]
    for utterance in sentences:
        langs.append(ft_model.predict(utterance)[0][0][-2:] )

    if np.unique(langs).shape[0] > 1:
        #Check for utterances translated to another languages
        e_sentences = embedd(sentences)
        cobinations_i = list(itertools.combinations(range(len(e_sentences)), r=2))
        utts_to_delete = []
        for i_ in cobinations_i:
            if all(e_sentences[i_[0]] != e_sentences[i_[1]]):
                c_similarity = np.matmul(e_sentences[i_[0]], np.transpose(e_sentences[i_[1]] ) )
#                 print('-',langs[i_[0]], sentences[i_[0]],"\n-",langs[i_[1]],sentences[i_[1]],"\n",   c_similarity,"\n","\n" )
                if langs[i_[0]] != langs[i_[1]] and c_similarity >= 0.8: #WARNING ARBITRARY TRESHOLD!
#                     print(sentences[i_[0]],"\n",sentences[i_[1]])
                    if langs[i_[0]] != lang_model.get_dominant_language():
                        delete_utt = sentences[i_[0]]
                    elif langs[i_[1]] != lang_model.get_dominant_language():
                        delete_utt = sentences[i_[1]]
                    utts_to_delete.append(delete_utt)
        if len(utts_to_delete) > 0:
            [sentences.remove(x) for x in utts_to_delete]

            
    lemm_sentences = []
    ents = []
    pos = []
    sentences = [{'utterance':utt,'language':lang} for utt,lang in zip(sentences,langs) if len(utt.split(" "))>1]
    for utterance in sentences:
        nlp_core = lang_model.get_nlp_core(utterance['language'])
        stopwords = lang_model.get_stopwords(utterance['language'])
        if nlp_core and stopwords:
            utt = nlp_core(utterance['utterance'])
            utt_lem = [token.lemma_ for token in utt 
                       if token.lemma_ not in stopwords and not token.lemma_.isspace()]
        
#             for token in utt:
#                 print(token.text, token.lemma_, token.pos_, token.dep_)
            ent_ = [{'ent' : " ".join([t.lemma_ for t in nlp_core(ent.text)]), 
                          'type' : ent.label_ } for ent in utt.ents if not (len(ent.text.split(' '))>1 and ent.label_ == 'MISC')]
            ent_ = [ f"{e['ent']}-{e['type']}" if ent_ else "Nan" for e in ent_ ]
            ent_ = ', '.join(ent_)
            ents.append(ent_)
            
            utt_lem = ', '.join(utt_lem)
            lemm_sentences.append(utt_lem)
                    
            pos_ =[token.lemma_ for token in utt if token.pos_ == 'VERB']
            pos_ = ', '.join(pos_)        
            pos.append(pos_)        
        else:
            pos.append("Nan")
            ents.append("Nan")
            lemm_sentences.append("Nan")
    ents = [i if i else 'Nan' for i in ents]
    if ents:
        ents = '; '.join(ents)
    if pos:
        pos = '; '.join(pos)
    if lemm_sentences:
        lemm_sentences = '; '.join(lemm_sentences)

    return {"lemm_sentences": lemm_sentences,"ents": ents, "pos":pos }



In [169]:
for country in countries:
    country='GB.1'
    print(country)
    lang_model = Language_Models(country)
    lang_frame_uk = pd.read_excel(open(name_uk, 'rb'),sheet_name = country)
    if lang_frame_uk.columns.shape[0] == 0:
        print("no samples")
        continue
    fix_names_uk = lang_frame_uk.columns
    lang_frame_uk.set_axis(fix_names, axis=1,inplace=True)
    lang_frame_uk.loc[0] = fix_names_uk
    lang_frame_uk = lang_frame_uk[["status_id","created_at","text","reply_to_status_id","profile_expanded_url"]]

    e = lang_frame_uk["text"].apply(lambda x: process(x,lang_model))
    lemm_sentences = [i["lemm_sentences"] for i in e]
    ents = [i["ents"] for i in e]
    pos = [i["pos"] for i in e]

    lang_frame_uk["lemm_sentences"] = lemm_sentences
    lang_frame_uk["ents"]=ents
    lang_frame_uk["pos"]=pos
#     lang_frame_uk['sentiment'] = lang_frame_uk['lemm_sentences'].apply(get_sentiment_vec)
    del lang_model
    
    break

GB.1
en_core_web_lg
nl_core_web_lg
ja_core_web_lg
no ja nlp core
fr_core_web_lg
vi_core_web_lg
no vi nlp core
ru_core_web_lg
es_core_web_lg
ds_core_web_lg
no ds nlp core
de_core_web_lg
ca_core_web_lg
no ca nlp core


In [177]:
lang_frame_uk['lemm_sentences'].apply(get_sentiment_vec)

0    [0.07142857142857142, 0.07142857142857142, 0.2...
1    [0.2857142857142857, 0.14285714285714285, 0.14...
2    [0.2, 0.2, 0.2, 0.06666666666666667, 0, 0.1333...
3    [0.1875, 0.1875, 0.125, 0.125, 0.125, 0.125, 0...
4    [0.14814814814814814, 0.07407407407407407, 0.1...
Name: lemm_sentences, dtype: object

In [89]:
en_sent_lex = pd.read_csv('lang_sentiment_dicts/en_sentiments.csv')
en_index_emb = np.genfromtxt('lang_sentiment_dicts/en_embedded.csv', delimiter=',')


In [90]:
en_sent_lex.shape[0]==en_index_emb.shape[0]

True

In [164]:

for twitt_lem, text in zip(lang_frame_uk.lemm_sentences,lang_frame_uk.text):
    sentences = twitt_lem.split('; ')
    for utterance in sentences:

        sentiment = []
        for word in utterance.split(', '):

            e1 = embedd(word)
            score = np.matmul(e1, np.transpose(en_index_emb))
            if max(score[0])>0.8:
                sentiment_tag = en_sent_lex.iloc[np.argmax(score[0])]
                print(word, sentiment_tag.lem, sentiment_tag.val,max(score[0]))
                emotions = sentiment_tag.val.split(',')
                sentiment.append(emotions)
        sentiment = np.array([item for sublist in sentiment for item in sublist])
        sentiment = val_ct_dict(sentiment)
        if sentiment.get("positive") and sentiment.get("negative"):
            vibe = (sentiment["positive"] - sentiment["negative"])/(sentiment["positive"] + sentiment["negative"])*100
            del sentiment["positive"]
            del sentiment["negative"]
        elif sentiment.get("positive"):
            vibe = 1
            del sentiment["positive"]
        elif sentiment.get("negative"):
            vibe = -1
            del sentiment["negative"]

        sentiment = dict([(i,j/sum(list(sentiment.values())) ) for i, j in sentiment.items()])
        sentiment_ret = [ sentiment.get(s) if sentiment.get(s) else 0 for s in emotions_list]
        return sentiment_ret
#         print(text)
#         print(emotions_list)
#         print(sentiment_ret)
#         print(sentiment)
#         print(vibe)
#         print("---------------------------")



join join positive 1.0000000251041672
Majesty majesty positive,trust 0.8323704198388557
Prince king positive 0.8032521887140971
visit visit positive 1.0000000037499017
Donation donation positive 0.8847430031121892
Centre center positive,trust 0.9141388795714376
excellent superb positive 0.936002075166009
response react anger,fear 0.8516560344122279
authority authoritative positive,trust 0.8343603331184499
local localize anticipation 0.8073445661351674
organisation organization anticipation,joy,positive,surprise,trust 0.9949117931883841
humanitarian humanitarian anticipation,joy,positive,surprise,trust 1.0000000093619528
crisis crisis negative 1.0000000158635025
🇺🇦🇷🇴🇬🇧 Joined by Her Majesty Margareta of Romania, The Prince of Wales has visited the Romexpo Donation Centre in Bucharest to see first-hand the excellent response of the Romanian authorities and international and local organisations to the humanitarian crisis in Ukraine. https://t.co/v92d50hfA7
['fear', 'anger', 'anticipation'

In [ ]:
lang_frame_uk[["lemm_sentences","ents", "pos"]] = e

In [ ]:
lemm_sentences = [i["lemm_sentences"] for i in e]
ents = [i["ents"] for i in e]
pos = [i["pos"] for i in e]

lang_frame_uk["lemm_sentences"] = lemm_sentences
lang_frame_uk["ents"]=ents
lang_frame_uk["pos"]=pos

In [ ]:
lang_frame_uk

In [ ]:
e1 = embedd("duck")
e2 = embedd("pond")
np.matmul(e1, np.transpose(e2) )

In [ ]:
e1 = embedd("kaczka")
e2 = embedd("staw")
np.matmul(e1, np.transpose(e2) )

In [ ]:
e1 = embedd("kaczka")
e2 = embedd("bird")
np.matmul(e1, np.transpose(e2) )